In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import json
from time import time
import numpy as np
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, ngrams

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import DictVectorizer 
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SVMSMOTE, BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,mutual_info_classif
from sklearn.svm import SVC, libsvm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import numpy as np
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.linear_model import LogisticRegressionCV
from operator import itemgetter
from random import shuffle


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.libsvm module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from skle

In [ ]:
pooling_function = 'avg'
with open('/content/drive/My Drive/'+pooling_function+'-labels-intense-selection.json', 'r', encoding='utf-8') as file:
    data = json.load(file)



In [ ]:
good = sorted([x for x in data if x[1] == 1], key=itemgetter(2))
bad = sorted([x for x in  data if x[1] == -1], key=itemgetter(2))


In [ ]:
data = good[-10000:] + bad[-10000:]
shuffle(data)

In [ ]:
len(data)

20000

In [ ]:
word_freq = {}
with open('/content/drive/My Drive/unigram_freq.csv', 'r', encoding='utf-8') as file:
    count = 1
    for line in file:
      line  = line.strip().split(',')
      word_freq[line[0]] = count
      count += 1
      if count == 17000:
        break

In [ ]:
def tokenize(string):
  return string.split()

y = np.array([x[1] for x in data])
print(len(y))

start = time()
n1gram = 1
n2gram = 1
#v = DictVectorizer(sparse=True)
#v = CountVectorizer(ngram_range=(n1gram,n2gram))
v = TfidfVectorizer(ngram_range=(n1gram,n2gram), lowercase=False, sublinear_tf=True, tokenizer=tokenize, stop_words='english')
docs = [x[0] + ' '.join([j[1] for j in nltk.pos_tag(x[0].split())]) for x in data]
X_ = v.fit_transform(docs)
end = time()
tfidf_scores = v.vocabulary_end = time()

20000


In [ ]:
X_.shape

(20000, 52318)

Naive Bayes

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = MultinomialNB()

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852
classification time 0.0108
TPR Recall 0.736
TNR specificity 0.6715
precision 0.6913
fmeasure 0.7129
accuracy 0.7038
bal acc 0.7038
---------------------------------------------
0.736 0.6715 0.6913 0.7129 0.7038 0.7038
---------------------------------------------
tp 736.0
fp 328.5
fn 264.0
tn 671.5


Random Forest

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = RandomForestClassifier(n_estimators=300)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852
classification time 241.0152
TPR Recall 0.8311
TNR specificity 0.6736
precision 0.7181
fmeasure 0.7704
accuracy 0.7523
bal acc 0.7523
---------------------------------------------
0.8311 0.6736 0.7181 0.7704 0.7523 0.7523
---------------------------------------------
tp 831.1
fp 326.4
fn 168.9
tn 673.6


In [ ]:
SVC

sklearn.svm._classes.SVC

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = SVC(max_iter=10000)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/s

classification time 90.2249
TPR Recall 0.7767
TNR specificity 0.7145
precision 0.7313
fmeasure 0.7532
accuracy 0.7456
bal acc 0.7456
---------------------------------------------
0.7767 0.7145 0.7313 0.7532 0.7456 0.7456
---------------------------------------------
tp 776.7
fp 285.5
fn 223.3
tn 714.5


SGD

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = SGDClassifier(max_iter=2000, tol=1e-5)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852
classification time 0.6455
TPR Recall 0.7788
TNR specificity 0.7159
precision 0.7328
fmeasure 0.755
accuracy 0.7474
bal acc 0.7474
---------------------------------------------
0.7788 0.7159 0.7328 0.755 0.7474 0.7474
---------------------------------------------
tp 778.8
fp 284.1
fn 221.2
tn 715.9


KNN

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = KNeighborsClassifier(n_neighbors=200)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852
classification time 2.0798
TPR Recall 0.7218
TNR specificity 0.6731
precision 0.6884
fmeasure 0.7046
accuracy 0.6974
bal acc 0.6974
---------------------------------------------
0.7218 0.6731 0.6884 0.7046 0.6974 0.6974
---------------------------------------------
tp 721.8
fp 326.9
fn 278.2
tn 673.1


Logistic Regression

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = LogisticRegressionCV()

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


classification time 36.8125
TPR Recall 0.7574
TNR specificity 0.7334
precision 0.7397
fmeasure 0.7483
accuracy 0.7454
bal acc 0.7454
---------------------------------------------
0.7574 0.7334 0.7397 0.7483 0.7454 0.7454
---------------------------------------------
tp 757.4
fp 266.6
fn 242.6
tn 733.4


LSVC

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10

clf = LinearSVC(random_state=0, tol=1e-10, loss='hinge')

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 16.852
classification time 0.3794
TPR Recall 0.7699
TNR specificity 0.7205
precision 0.7337
fmeasure 0.7513
accuracy 0.7452
bal acc 0.7452
---------------------------------------------
0.7699 0.7205 0.7337 0.7513 0.7452 0.7452
---------------------------------------------
tp 769.9
fp 279.5
fn 230.1
tn 720.5
